In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ipywidgets import widgets
import math as math
import itertools

import warnings; warnings.simplefilter("ignore")
from IPython.display import display, clear_output

import bokeh.charts as charts
import bokeh.layouts as layouts
import bokeh.plotting as plotting
charts.output_notebook()

Loading BokehJS ...

### Data preparation

In [10]:
raw = pd.read_csv("loan.csv")

In [11]:
data = raw.iloc[:, [2, 5, 6, 7, 8, 11, 12, 13, 16, 33, 32]]
data.head()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,loan_status,revol_util,revol_bal
0,5000.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Fully Paid,83.7,13648.0
1,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30000.0,Charged Off,9.4,1687.0
2,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Fully Paid,98.5,2956.0
3,10000.0,36 months,13.49,339.31,C,10+ years,RENT,49200.0,Fully Paid,21.0,5598.0
4,3000.0,60 months,12.69,67.79,B,1 year,RENT,80000.0,Current,53.9,27783.0


In [12]:
for column in data.columns:
    data.loc[:, column].fillna(0, inplace=True)
data.dtypes

loan_amnt         float64
term               object
int_rate          float64
installment       float64
grade              object
emp_length         object
home_ownership     object
annual_inc        float64
loan_status        object
revol_util        float64
revol_bal         float64
dtype: object

In [13]:
data["annual_inc_over_average"] = (data["annual_inc"] > data["annual_inc"].mean()).apply(lambda val: "Yes" if val else "No")
for column in data.columns:
    if data[column].dtype != np.int64 and data[column].dtype != np.float64:
        if data[column].dtype == np.bool:
            data.loc[:, column] = data[column].astype("category")
        else:
            data.loc[:, column] = data[column].str.strip().astype("category")

### Some helpers

In [14]:
class Dropdown(widgets.Dropdown):
    def __init__(self, options, initial_value, source, renderer, description=None):
        self.source = source
        self.renderer = renderer
        base = super(Dropdown, self)
        base.__init__(
            options=options,
            value=initial_value,
            description=description,
        )
        base.on_displayed(self._on_displayed)
        base.observe(self._on_changed, names="value")

    def _on_displayed(self, arg):
        value = arg.value
        clear_output()
        self.renderer(self.source[value])

    def _on_changed(self, arg):
        value = arg["new"]
        clear_output()
        self.renderer(self.source[value])

In [15]:
def compute_proportion_ci(prop, n, conf_level, dist="z"):
    pp = conf_level/100  + (1-conf_level/100)/2
    if dist == "z":
        stat = stats.norm.ppf(pp)
    else:
        stat = stats.t.ppf(pp, n-1)
    try:
        return [stat * math.sqrt((p*(1-p)/n)) for p in prop]
    except TypeError:
        return stat * math.sqrt((prop*(1-prop)/n))
    
def compute_mean_ci(n, stdev, conf_level, dist="z"):
    pp = conf_level/100  + (1-conf_level/100)/2
    if dist == "z":
        stat = stats.norm.ppf(pp)
    else:
        stat = stats.t.ppf(pp, n-1)
    return stat * stdev / math.sqrt(n)

def compute_stdev_ci(stdev, n, conf_level):
    pp = conf_level/100  + (1-conf_level/100)/2
    df = n - 1
    lower_stat, upper_stat = stats.chi2.ppf(1-pp, df), stats.chi2.ppf(pp, df)
    return math.sqrt((n-1)*(stdev**2)/lower_stat), math.sqrt((n-1)*(stdev**2)/upper_stat)

def compute_prop_diff_ci(p1, p2, n1, n2, conf_level):
    pp = conf_level/100  + (1-conf_level/100)/2
    stat = stats.norm.ppf(pp)
    return stat * math.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)

### Relative frequency tables

In [16]:
def bins_to_ranges(bins):
    ranges = []
    for i in range(len(bins)-1):
        ranges.append((bins[i], bins[i+1]))
    return ranges

def reject_outliers(seq, rng):
    iqr = stats.iqr(seq, rng=(100-rng, rng), scale='raw')
    median = seq.median()
    return seq[lambda x: (x > (median - iqr/2)) & (x < (median + iqr/2))]

dfs = {}

n = data.shape[0]
for column in data.columns:
    df: pd.DataFrame
    if data[column].dtype == np.int64 or data[column].dtype == np.float64:
        col = reject_outliers(data[column], 99.5)
        hist, bins = np.histogram(col)
        ranges = bins_to_ranges(bins)
        df = pd.DataFrame(index=pd.Index(data=ranges, name=column, tupleize_cols=False))
        df["Frequency"] = hist
    else:
        freq = data[column].groupby(data[column]).size()
        df = pd.DataFrame(index=pd.Index(data=freq.index.categories, name=column))
        df["Frequency"] = freq
        df.sort_values("Frequency", ascending=False, inplace=True)
    df["Proportion"] = df["Frequency"] / n
    df["95% CI"] = compute_proportion_ci(df["Proportion"], n, 95)
    df["Percent"] = df["Frequency"] * (100 / n)
    dfs[column] = df

tables_dropdown = Dropdown(
    options=data.columns.tolist(),
    initial_value=data.columns[6],
    source=dfs,
    renderer=display,
    description="Relative frequency tables"
)

tables_dropdown

,Frequency,Proportion,95% CI,Percent
home_ownership,,,,
MORTGAGE,103500,0.511030,0.002177,51.103036
RENT,82316,0.406435,0.002139,40.643454
OWN,16485,0.081395,0.001191,8.139455
OTHER,181,0.000894,0.000130,0.089369
NONE,50,0.000247,0.000068,0.024687


### Histograms

In [17]:
def make_bar_graph(freq, ci=None):
    relative_frequencies = freq / freq.sum()
    if isinstance(freq.index[0], tuple):
        index = [f"{format(a,'.1f')} - {format(b,'.1f')}" for a, b in freq.index]
    else:
        index = freq.index.tolist()
    p = plotting.figure(x_range=index, title=f"{freq.index.name} histogram", plot_width=800)
    #p.sizing_mode = "scale_width"
    p.yaxis.axis_label = "Relative frequencies"
    p.xaxis.axis_label = relative_frequencies.index.name
    p.xaxis.major_label_orientation = np.pi/4
    p.vbar(x=index, width=0.5, bottom=0, top=relative_frequencies.tolist())
    if ci is not None:
        y0, y1 = zip(*[(f-c, f+c) for f, c in zip(freq, ci)])
        p.segment(x0=index, x1=index, y0=y0, y1=y1)
    return p

histograms = {column: make_bar_graph(df["Frequency"], df["95% CI"]) for column, df in dfs.items()}

histograms_dropdown = Dropdown(
    options=data.columns.tolist(),
    initial_value=data.columns[6],
    source=histograms,
    renderer=charts.show,
    description="Histograms"
)

histograms_dropdown

### Relative frequency line graphs

In [18]:
def make_frequency_line_graph(frequencies):
    if isinstance(frequencies.index[0], tuple):
        sorted_freq = frequencies.sort_index(ascending=True)
        index = [f"{format(a,'.1f')} - {format(b,'.1f')}" for a, b in sorted_freq.index]
        relative_frequencies = sorted_freq / frequencies.sum()
    else:
        sorted_freq = frequencies.sort_values(ascending=True)
        index = sorted_freq.index.tolist()
        relative_frequencies = sorted_freq / frequencies.sum()
    p = plotting.figure(x_range=index, title=f"{frequencies.index.name} frequency line graph", plot_width=800)
    #p.sizing_mode = "scale_width"
    p.yaxis.axis_label = "Relative frequencies"
    p.xaxis.axis_label = relative_frequencies.index.name
    p.xaxis.major_label_orientation = np.pi/4
    p.line(x=index, y=relative_frequencies, line_width=2, legend="Relative frequencies")
    p.line(x=index, y=np.cumsum(relative_frequencies), line_width=2, color="red", legend="Cumulative relative frequencies")
    p.legend.location = "top_left"
    return p

line_graphs = {column: make_frequency_line_graph(df["Frequency"]) for column, df in dfs.items()}

line_graphs_dropdown = Dropdown(
    options=data.columns.tolist(),
    initial_value=data.columns[6],
    source=line_graphs,
    renderer=charts.show,
    description="Relative frequency line graphs"
)

line_graphs_dropdown

### Some statistics

In [19]:
vars = data.select_dtypes(include=[np.number]).columns
statistics = ["Mean", "Median", "Mode", "Variance", "Standard deviation", "P(within 1.5 std)"]
n = data.shape[0]

def compute_column_stats(col):
    mean = col.mean()
    mode, _ = stats.mode(col)
    median = col.median()
    std = np.std(col)
    var = std ** 2
    lower, upper = mean-1.5*std, mean+1.5*std
    p = col[(col > lower) & (col < upper)].size / n
    return mean, median, mode, var, std, p

stat_data = np.transpose([compute_column_stats(data[col]) for col in vars])
stats_df = pd.DataFrame(data=stat_data, index=statistics, columns=vars)

stats_df

,loan_amnt,int_rate,installment,annual_inc,revol_util,revol_bal
Mean,1.382774e+04,13.982326,422.823952,7.211207e+04,56.600963,1.616375e+04
Median,1.200000e+04,13.680000,375.380000,6.200000e+04,58.700000,1.191150e+04
Mode,1.000000e+04,12.120000,332.720000,6.000000e+04,0.000000,0.000000e+00
Variance,6.538564e+07,19.218313,58033.334192,3.106423e+09,590.575858,4.238922e+08
Standard deviation,8.086139e+03,4.383870,240.901088,5.573529e+04,24.301767,2.058864e+04
P(within 1.5 std),9.019118e-01,0.866105,0.898500,9.621689e-01,0.865666,9.682421e-01


To apply the "empirical rule", we assume a normal distribution and compute the z-scores for $X_1=\mu-1.5\sigma$ and $X_2=\mu+1.5\sigma$. We then use the z-scores to compute $\int_{z_1}^{z_2} \varphi(x)dx$. The computation of the z-scores can be bypassed by computing the P-values directly relative to the sample mean and standard deviation:

In [20]:
p_lower, p_upper = stats.norm.cdf([-1.5, 1.5])
empirical = pd.Series(data=[p_upper - p_lower, 1 - 1/(1.5 ** 2)], index=pd.Index(["Empirical", "Chebyshev (lower bound)"]), name="P(within 1.5 std)")
empirical

Empirical                  0.866386
Chebyshev (lower bound)    0.555556
Name: P(within 1.5 std), dtype: float64

The empirical rule provides good estimates for all six variables except the annual income. The Chebyshev inequality provides fairly poor estimates for all six variables.

### Error statistics

We use the Student-t distribution to obtain the confidence interval for the mean and the Chi-Square distribution for obtain the confidence interval for the standard deviation.

In [21]:
errors = pd.DataFrame(index=pd.MultiIndex.from_product([["Mean", "Standard deviation"], ["Value", "95% CI"]]), columns=vars)

for column in errors.columns:
    errors.loc[("Mean", "Value"), column] = stats_df.loc["Mean", column]
    errors.loc[("Mean", "95% CI"), column] = compute_mean_ci(n, stats_df.loc["Standard deviation", column], 95, "t")
    errors.loc[("Standard deviation", "Value"), column] = stats_df.loc["Standard deviation", column]
    errors.loc[("Standard deviation", "95% CI"), column] = compute_stdev_ci(stats_df.loc["Standard deviation", column], n, 95)

errors

loan_amnt  \
Mean               Value                                  13827.7   
                   95% CI                                 35.2164   
Standard deviation Value                                  8086.14   
                   95% CI  (8111.117903065599, 8061.314117524278)   

                                                         int_rate  \
Mean               Value                                  13.9823   
                   95% CI                               0.0190924   
Standard deviation Value                                  4.38387   
                   95% CI  (4.397411993305335, 4.370411058727791)   

                                                       installment  \
Mean               Value                                   422.824   
                   95% CI                                  1.04916   
Standard deviation Value                                   240.901   
                   95% CI  (241.6452619529038, 240.16151471273233)   

                                                       annual_inc  \
Mean               Value                                  72112.1   
                   95% CI                                 242.736   
Standard deviation Value                                  55735.3   
                   95% CI  (55907.46449451847, 55564.18217448319)   

                                                         revol_util  \
Mean               Value                                     56.601   
                   95% CI                                  0.105838   
Standard deviation Value                                    24.3018   
                   95% CI  (24.376837802843742, 24.227159445727324)   

                                                         revol_bal  
Mean               Value                                   16163.8  
                   95% CI                                  89.6668  
Standard deviation Value                                   20588.6  
                   95% CI  (20652.24232158292, 20525.433679442136)

### Contingency tables (observed)

In [22]:
def make_contingency_table(col1, col2):  
    probabilities = pd.crosstab(col1, col2)
    probabilities = probabilities/n
    #remove categories
    probabilities.index = pd.Index(probabilities.index.categories, name=probabilities.index.name)
    probabilities.columns = pd.Index(probabilities.columns.categories, name=probabilities.columns.name)
    probabilities = probabilities.append(probabilities.sum(axis=0).rename(f"P({col2.name})"))
    probabilities[f"P({col1.name})"] = probabilities.sum(axis=1).rename(f"P({col1.name})")
    return probabilities

categorical_columns = data.select_dtypes(exclude=[np.number]).columns
options = [f"{col2} vs {col1}" for col1, col2 in itertools.combinations(categorical_columns, 2)]

obs_contingency_tables = [make_contingency_table(data[col1], data[col2]) for col1, col2 in itertools.combinations(categorical_columns, 2)]

obs_contingency_tables_dict = {key: val for key, val in zip(options, obs_contingency_tables)}

obs_contingency_tables_dropdown = Dropdown(
    options=options,
    initial_value=options[0],
    source=obs_contingency_tables_dict,
    renderer=display,
    description="Contingency tables (observed)"
)

obs_contingency_tables_dropdown

grade,A,B,C,D,E,F,G,P(term)
term,,,,,,,,
36 months,0.152998,0.287708,0.177883,0.103653,0.024307,0.005273,0.001190,0.753012
60 months,0.005555,0.035486,0.082994,0.045914,0.044763,0.026105,0.006172,0.246988
P(grade),0.158553,0.323193,0.260877,0.149566,0.069071,0.031378,0.007362,1.000000


### Chi-square independence tests

In [23]:
indep_tests = pd.DataFrame(index=[options], columns=["Chi2 statistic", "P-value"])
expected_frequencies = {}
#options = [key for key in obs_contingency_tables_dict]

for name, table in obs_contingency_tables_dict.items():
    statistic, p_value, _, exp = stats.chi2_contingency(table * n)
    indep_tests.loc[name] = statistic, p_value
    expected_frequencies[name] = exp

display(indep_tests)

,Chi2 statistic,P-value
grade vs term,43000.8,0
emp_length vs term,1896.01,0
home_ownership vs term,2939.87,0
loan_status vs term,7789.34,0
annual_inc_over_average vs term,1614.49,0
emp_length vs grade,245.026,1.14453e-17
home_ownership vs grade,3003.47,0
loan_status vs grade,11620.2,0
annual_inc_over_average vs grade,2522.99,0
home_ownership vs emp_length,10886.5,0


Conclusion: none of these variables are independent. Since the P-values are all essentially zero, the null hypothesis of independence is rejected.

### Contingency tables (expected from independence hypothesis)

In [24]:
exp_contingency_tables_dict = {key: pd.DataFrame(data=expected_frequencies[key] / n, index=val.index.tolist(), columns=val.columns.tolist()) 
                               for key, val in obs_contingency_tables_dict.items()}
options = [key for key in exp_contingency_tables_dict]
exp_contingency_tables_dropdown = Dropdown(
    options=options,
    initial_value=options[0],
    source=exp_contingency_tables_dict,
    renderer=display,
    description="Contingency tables (expected)"
)

exp_contingency_tables_dropdown

,A,B,C,D,E,F,G,P(term)
36 months,0.119392,0.243368,0.196444,0.112625,0.052011,0.023628,0.005544,0.753012
60 months,0.039161,0.079825,0.064434,0.036941,0.017060,0.007750,0.001818,0.246988
P(grade),0.158553,0.323193,0.260877,0.149566,0.069071,0.031378,0.007362,1.000000


### Linear regression statistics

In [25]:
continuous_columns = data.select_dtypes(include=[np.int64, np.float64]).columns
continuous_columns_pairs = [pair for pair in itertools.combinations(continuous_columns, 2)]
options = [f"{col2} vs {col1}"  for col1, col2 in continuous_columns_pairs]

def get_lin_reg_row(col1, col2):
    slope, yint, cor, _, _ = stats.linregress(col1, col2)
    return slope, yint, cor

lin_reg_table_data = [get_lin_reg_row(data[col1], data[col2]) for col1, col2 in continuous_columns_pairs]
lin_reg_table = pd.DataFrame(data=lin_reg_table_data, index=options, columns=["slope", "y-int", "cor"])

lin_reg_table

,slope,y-int,cor
int_rate vs loan_amnt,0.000109,12.470890,0.201615
installment vs loan_amnt,0.028331,31.064754,0.950978
annual_inc vs loan_amnt,2.336340,39805.775264,0.338959
revol_util vs loan_amnt,0.000353,51.721807,0.117408
revol_bal vs loan_amnt,0.780313,5373.791564,0.306466
installment vs int_rate,10.358875,277.982787,0.188509
annual_inc vs int_rate,-234.513082,75391.112316,-0.018446
revol_util vs int_rate,2.142366,26.645707,0.386468
revol_bal vs int_rate,30.050723,15743.572745,0.006399
annual_inc vs installment,78.121897,39080.264632,0.337661


Conclusion: most of the variables have weak linear correlation to each other.

### Scatter plots

In [26]:
def make_linear_regression_plot(col1, col2):
    no_outliers1 = reject_outliers(col1, 95)
    no_outliers2 = reject_outliers(col2, 95)
    index_intersect = np.intersect1d(no_outliers1.index, no_outliers2.index, assume_unique=True)
    sample = np.random.choice(index_intersect, 1000, replace=False)
    title = f"{col2.name} vs {col1.name}"
    p = plotting.figure(title=title, plot_width=800)
    x_sample = col1.iloc[sample]
    y_sample = col2.iloc[sample]
    p.circle(x_sample, y_sample)
    slope, yint = lin_reg_table.loc[title, "slope"], lin_reg_table.loc[title, "y-int"]
    x_lin = np.linspace(start=x_sample.min(), stop=x_sample.max(), num=100)
    y_lin = np.apply_along_axis(lambda val: slope*val + yint, 0, x_lin)
    p.line(x_lin, y_lin, line_width=2, color="red", legend="Linear fit")
    p.yaxis.axis_label = col2.name
    p.xaxis.axis_label = col1.name
    p.legend.location = "top_left" 
    return p

scatter_plots = [make_linear_regression_plot(data[col1], data[col2]) for col1, col2 in itertools.combinations(continuous_columns, 2)]
scatter_plots_dict = {key: val for key, val in zip(options, scatter_plots)}

scatter_plots_dropdown = Dropdown(
    options=options,
    initial_value=options[0],
    source=scatter_plots_dict,
    renderer=charts.show,
    description="Scatter plots"
)

scatter_plots_dropdown

### Some hypothesis testing

Let's compare the rates of default for customers with income below the mean and customers with income above the mean. We define default as any loan status other than current, in grace period, or fully paid.

In [27]:
below = data["loan_status"].groupby(data["annual_inc_over_average"]).get_group("No")
above = data[~data.index.isin(below.index)]["loan_status"]

p_default_below = below[~below.str.contains("Paid|Current|Grace")].size / n
p_default_above = above[~above.str.contains("Paid|Current|Grace")].size / n
p_default_below, p_default_above

(0.094859083996603, 0.039850492761637664)

It appears that the rate of default is higher for customers with income below the mean. How significant is this result?

In [28]:
def z_stat_diff_prop(p1, p2, d, n1, n2):
    z_stat = (p1 - p2 - d)/math.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/p2)
    return z_stat

diff = p_default_below-p_default_above
z_stat = z_stat_diff_prop(p_default_below, p_default_above, diff, below.size, above.size)
p_value = stats.norm.sf(z_stat)
z_stat, p_value

(0.0, 0.5)

Conclusion: the result is very significant. We can definitively state that customer with income above the mean are more likely to default on a loan.

How different are the rates of default for customers with income below the mean and customers with income above the mean?

In [29]:
ci = compute_prop_diff_ci(p_default_below, p_default_above, below.size, above.size, 90)
f"{diff} +/- {ci}"

'0.05500859123496534 +/- 0.0017882256584423391'

Let's compare the rates of default for customers who own a home and customers who do not. We define default as any loan status other than current, in grace period, or fully paid. Carrying a mortgage is counted as owning a home.

In [30]:
owns = pd.concat([data["loan_status"].groupby(data["home_ownership"]).get_group(group) for group in ["MORTGAGE", "OWN"]])
does_not_own = data[~data.index.isin(owns.index)]["loan_status"]

p_default_does_not_own = does_not_own[~does_not_own.str.contains("Paid|Current|Grace")].size / n
p_default_owns = owns[~owns.str.contains("Paid|Current|Grace")].size / n
p_default_does_not_own, p_default_owns

(0.06235064088637845, 0.07235893587186222)

It appears that the rate of default is higher for customers who own a home, which is somewhat surprising. How significant is this result?

In [31]:
diff = p_default_does_not_own - p_default_owns
z_stat = z_stat_diff_prop(p_default_does_not_own, p_default_owns, diff, does_not_own.size, owns.size)
p_value = stats.norm.sf(z_stat)
z_stat, p_value

(0.0, 0.5)

Conclusion: The evidence in favor of the result is very significant. We can definitively state that the rate of default is higher for customers who own a home compared to those who do not.

How different are the rates of default for customers who own a home and those who do not?

In [32]:
ci = compute_prop_diff_ci(p_default_does_not_own, p_default_owns, does_not_own.size, owns.size, 99.9)
f"{diff} +/- {ci}"

'-0.010008294985483772 +/- 0.0037048301497559074'